In [1]:
import librosa
import numpy as np
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [2]:
import librosa
import numpy as np
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Set desired common sample rate
common_sample_rate = 16000

# Path to the folder containing the audio files
fake_audio_path = './sounds/'

data = []
target = []

# Set a fixed duration for padding or trimming
fixed_duration = 10 # in seconds

for file in os.listdir(fake_audio_path):
    filepath = os.path.join(fake_audio_path, file)
    audio, sr = librosa.load(filepath, sr=None) # load the native sample rate
    
    # Resample audio if sample rate is different
    if sr != common_sample_rate:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=common_sample_rate)
    
    # Pad or trim the audio to a fixed duration
    audio = pad_sequences([audio], maxlen=common_sample_rate*fixed_duration, padding='post', truncating='post')[0]
    
    data.append(audio)
    target.append(1)  # 1 denotes fake audio

# Convert lists to numpy arrays
data = np.array(data)
target = np.array(target)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=50)

In [4]:
X_train = X_train.reshape(X_train.shape[0], -1)  # Flatten the input
X_test = X_test.reshape(X_test.shape[0], -1)


In [5]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)


RandomForestClassifier()

In [6]:
from sklearn.metrics import classification_report

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        15

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15



In [7]:
%pip install soundfile

DEPRECATION: Loading egg at c:\users\hp\appdata\local\programs\python\python311\lib\site-packages\tortoise-2.8.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [8]:
import numpy as np

# Generate random audio data
duration = 100  # 10 seconds
sample_rate = 19000
num_samples = duration * sample_rate
audio = np.random.uniform(low=-0.5, high=0.5, size=num_samples)

# Trim or pad the audio to match the desired length
desired_length = len(X_train[0])  # Assuming X_train has the same feature length as the classifier expects
if len(audio) < desired_length:
    audio = np.pad(audio, (0, desired_length - len(audio)), mode='constant')
elif len(audio) > desired_length:
    audio = audio[:desired_length]

audio = audio.reshape(1, -1)  # Reshape the audio for prediction

# Predict using the trained model
prediction = clf.predict(audio)
if prediction[0] == 1:
    print("Fake audio detected!")
else:
    print("Real audio detected!")


Fake audio detected!


In [9]:
import os
import torch
import torch.nn as nn

# Define your model class
class YourModelClass(nn.Module):
    def __init__(self):
        super(YourModelClass, self).__init__()

        # Define your model architecture here
        # ...

# Create an instance of your model
model = YourModelClass()

# Train your model...
# ...

# Specify the file path to save the model
model_path = 'C:\\Users\\hp\\Desktop\\tortoise-tts(major-proj)\\custommodel.pth'

# Check if the model file already exists
if os.path.exists(model_path):
    # Delete the old model file
    os.remove(model_path)

# Save the new model file
torch.save(model.state_dict(), model_path)

# Print a message to the user
print('The model has been saved to {}'.format(model_path))


The model has been saved to C:\Users\hp\Desktop\tortoise-tts(major-proj)\custommodel.pth


RNN

In [1]:
%pip install librosa torch

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\hp\appdata\local\programs\python\python311\lib\site-packages\fakenet_ng-3.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\hp\appdata\local\programs\python\python311\lib\site-packages\pydivert-2.1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\hp\appdata\local\programs\python\python311\lib\site-packages\pyftpdlib-1.5.9-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\hp\appdata\loc

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import librosa
import os
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Function to load and preprocess audio data with resampling, trimming, and flattening
def load_and_preprocess_audio(audio_file, target_sr=10000, max_duration=10, flatten=True):
    # Load audio file
    audio, sr = librosa.load(audio_file, sr=target_sr)

    # Resample if the sampling rate is higher than the target
    if sr > target_sr:
        audio = librosa.resample(audio, sr, target_sr)

    # Trim the audio if it exceeds the maximum duration
    if len(audio) > target_sr * max_duration:
        audio = audio[:target_sr * max_duration]

    # Flatten the audio if specified
    if flatten:
        audio = audio.flatten()

    return audio

# Set the path to your local sounds directory
sounds_directory = 'C:\\Users\\hp\\Desktop\\major-proj_Copy\\sounds'

# Load AI-generated voice dataset
X_train = []

for audio_file in os.listdir(sounds_directory):
    # Load and preprocess audio data
    audio_data = load_and_preprocess_audio(os.path.join(sounds_directory, audio_file))

    # Append audio data to training data
    X_train.append(audio_data)

# Set a fixed length for your sequences
max_len = 100
X_train_padded = torch.nn.utils.rnn.pad_sequence([torch.tensor(x) for x in X_train], batch_first=True)

# Reshape the input sequences to have a third dimension
X_train_padded = X_train_padded.unsqueeze(2)

# Split the data into train and test sets
X_train, X_test = train_test_split(X_train_padded, test_size=0.2, random_state=22)

# Define the RNN model using PyTorch
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output, _ = self.rnn(x)
        output = self.dropout(output)
        output = self.linear(output)
        return output

# Convert the data to PyTorch tensors
X_train = torch.nn.utils.rnn.pad_sequence([torch.tensor(x) for x in X_train], batch_first=True)
X_test = torch.nn.utils.rnn.pad_sequence([torch.tensor(x) for x in X_test], batch_first=True)

# Define the model, loss function, and optimizer
model = SimpleRNN(input_size=1, hidden_size=34, output_size=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train)
    loss = criterion(outputs, X_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Save the PyTorch model as a .pth file
torch.save(model.state_dict(), 'model.pth')


C:\Users\hp\AppData\Local\Temp\ipykernel_13536\2369643226.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.nn.utils.rnn.pad_sequence([torch.tensor(x) for x in X_train], batch_first=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_13536\2369643226.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.nn.utils.rnn.pad_sequence([torch.tensor(x) for x in X_test], batch_first=True)


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 11232083384 bytes.

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import librosa
import os
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

class AudioDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, target_sr=10000, max_duration=10, flatten=True, max_len=100000):
        self.data_dir = data_dir
        self.target_sr = target_sr
        self.max_duration = max_duration
        self.flatten = flatten
        self.max_len = max_len

        self.file_list = os.listdir(self.data_dir)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        audio_file = os.path.join(self.data_dir, self.file_list[idx])
        audio_data = self.load_and_preprocess_audio(audio_file)
        return torch.tensor(audio_data)

    def load_and_preprocess_audio(self, audio_file):
        audio, sr = librosa.load(audio_file, sr=self.target_sr)

        if sr > self.target_sr:
            audio = librosa.resample(audio, sr, self.target_sr)

        if len(audio) > self.target_sr * self.max_duration:
            audio = audio[:self.target_sr * self.max_duration]

        if self.flatten:
            audio = audio.flatten()

        # Pad or truncate to the specified max_len
        if len(audio) < self.max_len:
            audio = np.pad(audio, (0, self.max_len - len(audio)))
        else:
            audio = audio[:self.max_len]

        return audio

# Set the path to your local sounds directory
sounds_directory = 'C:\\Users\\hp\\Desktop\\major-proj_Copy\\sounds'

# Create custom dataset
audio_dataset = AudioDataset(data_dir=sounds_directory, max_len=100000)  # Use a suitable max_len

# Split the dataset into train and test sets
train_size = int(0.8 * len(audio_dataset))
test_size = len(audio_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(audio_dataset, [train_size, test_size])

# Use DataLoader for batching during training
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define the RNN model using PyTorch
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output, _ = self.rnn(x)
        output = self.dropout(output)
        output = self.linear(output)
        return output

# Convert the data to PyTorch tensors
def collate_fn(batch):
    return torch.stack(batch, dim=0).unsqueeze(2)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleRNN(input_size=1, hidden_size=34, output_size=1).to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    
    for batch_X in train_loader:
        batch_X = collate_fn(batch_X).to(device)
        outputs = model(batch_X)
        loss = criterion(outputs, batch_X)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Save the PyTorch model as a .pth file
torch.save(model.state_dict(), 'model.pth')


TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

: 

Link to the RNN model - https://colab.research.google.com/drive/1Lbi5enkautHGYgWaLYvRa9ZFaqlJS0Gt#scrollTo=i13THrjUgMNG